## 다채널 합성곱 연산 구현하기

In [1]:
import numpy as np
from numpy import ndarray

### 헬퍼 함수

In [2]:
def assert_same_shape(output: ndarray,
                      output_grad: ndarray):
  assert output.shape == output_grad.shape, \
  f"""
  두 ndarray의 모양이 같아야 하는데,
  첫 번째 ndarray의 모양은 {tuple(output_grad.shape)}이고,
  두 번째 ndarray의 모양은 {tuple(output.shape)}이다.
  """
  return None

In [3]:
def assert_dim(t: ndarray,
               dim: ndarray):
  assert len(t.shape) == dim, \
  f'이 텐서는 {dim}차원이어야 하는데, {len(t.shape)}차원이다.'
  
  return None

#### 1차원 합성곱

1. 우리의 목표는 입력과 크기가 같은 합서곱 연산의 출력을 계산하는 것이다.
2. 출력의 크기가 줄어드는 것을 방지하기 위해 입력에 패딩을 덧붙인다.
3. 필터로 입력을 훑어가며 각 위치에서 합성곱 연산을 반복하는 반복문을 작성한다.

패딩

In [4]:
input_1d = np.array([1,2,3,4,5])
param_1d = np.array([1,1,1])

In [5]:
def _pad_1d(inp: ndarray,
            num: int) -> ndarray:
  z = np.array([0])
  z = np.repeat(z, num)
  return np.concatenate([z, inp, z])

In [6]:
_pad_1d(input_1d, 1)

array([0, 1, 2, 3, 4, 5, 0])

순방향 계산

In [7]:
def conv_1d(inp: ndarray,
            param: ndarray) -> ndarray:
  
  # 입력과 파라미터의 차원 검증
  assert_dim(inp, 1)
  assert_dim(param, 1)

  # 입력에 패딩을 덧붙임
  param_len = param.shape[0]
  param_mid = param_len // 2
  input_pad = _pad_1d(inp, param_mid)

  # 출력값 초기화
  out = np.zeros(inp.shape)

  # 1차원 합성곱 연산 수행
  for o in range(out.shape[0]):
    for p in range(param_len):
      out[o] += param[p] * input_pad[o+p]

  # 출력의 모양이 입력과 동일한지 확인
  assert_same_shape(inp, out)

  return out

In [8]:
def conv_1d_sum(inp: ndarray,
                param: ndarray) -> ndarray:
  out = conv_1d(inp, param)
  return np.sum(out)

In [9]:
conv_1d_sum(input_1d, param_1d)

39.0

#### 합성곱 연산의 역방향 계산

1. 합성곱 연산의 입력(코드의 inp)의 각 요소에 대한 손실의 편미분을 계산한다.
2. 필터(코드의 param_id)의 각 요소에 대한 손실의 편미분을 계산한다.

정확한 기울기의 값 확인하기

In [10]:
# 5번째 요솟값 1 증가
input_1d_2 = np.array([1,2,3,4,6])
param_1d = np.array([1,1,1])

print(conv_1d_sum(input_1d, param_1d))
print(conv_1d_sum(input_1d_2, param_1d))

print('기울기: ',conv_1d_sum(input_1d_2, param_1d) - conv_1d_sum(input_1d, param_1d))

39.0
41.0
기울기:  2.0


파라미터 기울기
1. 입력과 필터를 인자로 받는다.
2. 출력을 계산한다.
3. 입력과 출력 기울기에 패딩을 덧붙인다.(input_pad와 output_pad)
4. 앞서 본 내용대로 패딩된 출력 기울기와 필터로 입력 기울기를 계산한다.
5. 같은 방법으로 패딩 안 된 출력 기울기와 패딩된 입력으로 필터 기울기를 계산한다.

In [11]:
def _param_grad_1d(inp: ndarray,
                   param: ndarray,
                   output_grad: ndarray = None) -> ndarray:
  param_len = param.shape[0]
  param_mid = param_len // 2
  input_pad = _pad_1d(inp, param_mid)

  if output_grad is None:
    output_grad = np.ones_like(inp)
  else:
    assert_same_shape(inp, output_grad)

  # 0으로 패딩된 1차원 합성곱
  param_grad = np.zeros_like(param)
  input_grad = np.zeros_like(inp)

  for o in range(inp.shape[0]):
    for p in range(param.shape[0]):
      param_grad[p] += input_pad[o+p] * output_grad[o]

  assert_same_shape(param_grad, param)

  return param_grad

In [12]:
def _input_grad_1d(inp: ndarray,
                   param: ndarray,
                   output_grad: ndarray = None) -> ndarray:
  param_len = param.shape[0]
  param_mid = param_len // 2
  inp_pad = _pad_1d(inp, param_mid)

  if output_grad is None:
    output_grad = np.ones_like(inp)
  else:
    assert_same_shape(inp, output_grad)

  output_pad = _pad_1d(output_grad, param_mid)

  # 0으로 패딩된 1차원 합성곱
  param_grad = np.zeros_like(param)
  input_grad = np.zeros_like(inp)

  for o in range(inp.shape[0]):
    for f in range(param.shape[0]):
      input_grad[o] += output_pad[o + param_len -f - 1] * param[f]

  assert_same_shape(param_grad, param)

  return input_grad

In [13]:
_input_grad_1d(input_1d, param_1d)

array([2, 3, 3, 3, 2])

In [14]:
_param_grad_1d(input_1d, param_1d)

array([10, 15, 14])

#### 배치 입력 적용하기
- 크기가 2인 배치

패딩

In [15]:
input_1d_batch = np.array([[0,1,2,3,4,5,6],
                           [1,2,3,4,5,6,7]])

In [16]:
def _pad_1d(inp: ndarray,
            num: int) -> ndarray:
  z = np.array([0])
  z = np.repeat(z, num)
  return np.concatenate([z, inp, z])

In [17]:
def _pad_1d_batch(inp: ndarray,
                  num: int) -> ndarray:
  outs = [_pad_1d(obs, num) for obs in inp]
  return np.stack(outs)

In [18]:
_pad_1d_batch(input_1d_batch, 1)

array([[0, 0, 1, 2, 3, 4, 5, 6, 0],
       [0, 1, 2, 3, 4, 5, 6, 7, 0]])

순방향 계산

In [19]:
def conv_1d_batch(inp: ndarray,
                  param: ndarray) -> ndarray:
  outs = [conv_1d(obs, param) for obs in inp]
  return np.stack(outs)

In [20]:
conv_1d_batch(input_1d_batch, param_1d)

array([[ 1.,  3.,  6.,  9., 12., 15., 11.],
       [ 3.,  6.,  9., 12., 15., 18., 13.]])

역방향 계산 - 기울기

In [21]:
def input_grad_1d_batch(inp: ndarray,
                        param: ndarray) -> ndarray:
  out = conv_1d_batch(inp, param)

  out_grad = np.ones_like(out)

  batch_size = out_grad.shape[0]

  grads = [_input_grad_1d(inp[i], param, out_grad[i]) for i in range(batch_size)]

  return np.stack(grads)

In [22]:
def param_grad_1d_batch(inp: ndarray,
                        param: ndarray) -> ndarray:
  
  output_grad = np.ones_like(inp)

  inp_pad = _pad_1d_batch(inp, 1)
  out_pad = _pad_1d_batch(inp, 1)

  param_grad = np.zeros_like(param)

  for i in range(inp.shape[0]):
    for o in range(inp.shape[1]):
      for p in range(param.shape[0]):
        param_grad[p] += inp_pad[i][o+p] * output_grad[i][o]

  return param_grad

conv_1d_batch로 계산한 기울기 검증

In [23]:
def conv_1d_batch_sum(inp: ndarray,
                      fil: ndarray) -> ndarray:
  out = conv_1d_batch(inp, fil)
  return np.sum(out)

In [24]:
conv_1d_batch_sum(input_1d_batch, param_1d)

133.0

In [25]:
print(np.random.randint(0, input_1d_batch.shape[0]))
print(np.random.randint(0, input_1d_batch.shape[1]))

0
1


In [26]:
input_1d_batch_2 = input_1d_batch.copy()
input_1d_batch_2[0][2] += 1
conv_1d_batch_sum(input_1d_batch_2, param_1d) - conv_1d_batch_sum(input_1d_batch, param_1d)

3.0

In [27]:
input_grad_1d_batch(input_1d_batch, param_1d)

array([[2, 3, 3, 3, 3, 3, 2],
       [2, 3, 3, 3, 3, 3, 2]])

In [28]:
print(np.random.randint(0, param_1d.shape[0]))

1


In [29]:
param_1d_2 = param_1d.copy()
param_1d_2[2] += 1
conv_1d_batch_sum(input_1d_batch, param_1d_2) - conv_1d_batch_sum(input_1d_batch, param_1d)

48.0

In [30]:
param_grad_1d_batch(input_1d_batch, param_1d)

array([36, 49, 48])

#### 2차원 합성곱

1. 순방향 계산 절차
  - 입력에 패딩을 덧붙인다.
  - 패딩된 입력과 파라미터로부터 출력을 계산한다.

2. 역방향 계산에서 입력 기울기를 계산하는 절차
  - 출력 기울기에 패딩을 덧붙인다.
  - 패딩된 출력 기울기와 입력, 파라미터로부터 입력 기울기와 파라미터 기울기를 계산한다.

3. 역방향 계산에서 파라미터 기울기를 계산하는 절차
  - 입력에 패딩을 덧붙인다.
  - 패딩된 입력의 각 요소를 순회하며 파라미터 기울기를 더한다.

In [31]:
imgs_2d_batch = np.random.randn(3, 28, 28)

In [32]:
param_2d = np.random.randn(3, 3)

패딩

In [33]:
def _pad_2d(inp: ndarray,
            num: int):
  """
  3차원 텐서를 입력받는다.
  첫 번째 차원은 배치 크기에 해당한다.
  """
  outs = [_pad_2d_obs(obs, num) for obs in inp]

  return np.stack(outs)

In [34]:
def _pad_2d_obs(inp: ndarray,
                num: int):
  """
  2차원이고 가로·세로 크기가 같은 텐서를 입력받는다.
  """
  inp_pad = _pad_1d_batch(inp, num)
  other = np.zeros((num, inp.shape[0] + num * 2))
  
  return np.concatenate([other, inp_pad, other])

In [35]:
_pad_2d(imgs_2d_batch, 1).shape

(3, 30, 30)

순방향 계산 - 출력값 계산

In [36]:
def _compute_output_obs_2d(obs: ndarray,
                           param: ndarray):
  """
  Obs는 2차원이고, 가로·세로 크기가 같은 텐서이며, 파라미터다.
  """
  param_mid = param.shape[0] // 2

  obs_pad = _pad_2d_obs(obs, param_mid)

  out = np.zeros_like(obs)

  for o_w in range(out.shape[0]):
    for o_h in range(out.shape[1]):
      for p_w in range(param.shape[0]):
        for p_h in range(param.shape[1]):
          out[o_w][o_h] += param[p_w][p_h] * obs_pad[o_w + p_w][o_h + p_h]

  return out

In [37]:
def _compute_output_2d(img_batch: ndarray,
                       param: ndarray):
  
  assert_dim(img_batch, 3)

  outs = [_compute_output_obs_2d(obs, param) for obs in img_batch]

  return np.stack(outs)

In [38]:
_compute_output_2d(imgs_2d_batch, param_2d).shape

(3, 28, 28)

역방향 계산 - 기울기

In [39]:
def _compute_grads_obs_2d(input_obs: ndarray,
                          output_grad_obs: ndarray,
                          param: ndarray) -> ndarray:
  """
  input_obs: 관찰을 나타내는 2차원 텐서
  output_grad_obs: 출력 기울기를 나타내는 2차원 텐서
  param: 2차원 필터
  """

  param_size = param.shape[0]
  output_obs_pad = _pad_2d_obs(output_grad_obs, param_size // 2)
  input_grad = np.zeros_like(input_obs)

  for i_w in range(input_obs.shape[0]):
    for i_h in range(input_obs.shape[1]):
      for p_w in range(param_size):
        for p_h in range(param_size):
          input_grad[i_w][i_h] += output_obs_pad[i_w + param_size - p_w - 1][i_h + param_size - p_h - 1] * param[p_w][p_h]

  return input_grad

In [40]:
def _compute_grads_2d(inp: ndarray,
                      output_grad: ndarray,
                      param: ndarray) -> ndarray:

  grads = [_compute_grads_obs_2d(inp[i], output_grad[i], param) for i in range(output_grad.shape[0])]

  return np.stack(grads)

In [41]:
def _param_grad_2d(inp: ndarray,
                   output_grad: ndarray,
                   param: ndarray) -> ndarray:
                  
  param_size = param.shape[0]
  inp_pad = _pad_2d(inp, param_size // 2)

  param_grad = np.zeros_like(param)
  img_shape = output_grad.shape[1:]

  for i in range(inp.shape[0]):
    for o_w in range(img_shape[0]):
      for o_h in range(img_shape[1]):
        for p_w in range(param_size):
          for p_h in range(param_size):
            param_grad[p_w][p_h] += inp_pad[i][o_w + p_w][o_h + p_h] * output_grad[i][o_w][o_h]

  return param_grad

In [42]:
img_grads = _compute_grads_2d(imgs_2d_batch,
                              np.ones_like(imgs_2d_batch),
                              param_2d)

In [43]:
img_grads.shape

(3, 28, 28)

In [44]:
param_grad = _param_grad_2d(imgs_2d_batch,
                            np.ones_like(imgs_2d_batch),
                            param_2d)

param_grad.shape

(3, 3)

기울기 검증

입력

In [45]:
print(np.random.randint(0, imgs_2d_batch.shape[0]))
print(np.random.randint(0, imgs_2d_batch.shape[1]))
print(np.random.randint(0, imgs_2d_batch.shape[2]))

2
19
17


In [46]:
imgs_2d_batch_2 = imgs_2d_batch.copy()
imgs_2d_batch_2[0][6][18] +=1

In [47]:
def _compute_output_2d_sum(img_batch: ndarray,
                           param: ndarray):
  out = _compute_output_2d(img_batch, param)

  return out.sum()

In [48]:
_compute_output_2d_sum(imgs_2d_batch_2, param_2d) - _compute_output_2d_sum(imgs_2d_batch, param_2d)

-1.3746262446754969

In [49]:
img_grads[0][6][18]

-1.3746262446754918

파라미터 기울기

In [50]:
print(np.random.randint(0, param_2d.shape[0]))
print(np.random.randint(0, param_2d.shape[1]))

0
2


In [51]:
param_2d_2 = param_2d.copy()
param_2d_2[0][2] += 1

In [52]:
_compute_output_2d_sum(imgs_2d_batch, param_2d_2) - _compute_output_2d_sum(imgs_2d_batch, param_2d)

20.801177388336242

In [53]:
param_grad[0][2]

20.80117738833626

#### 채널 추가하기

헬퍼 함수

In [54]:
def _pad_2d_channel(inp: ndarray,
                   num: int):
  """
  inp의 모양은 [num_channels, image_width, image_height]
  """

  return np.stack([_pad_2d_obs(channel, num) for channel in inp])

def _pad_conv_input(inp: ndarray,
                    num: int):
  """
  inp의 모양은 [batch_size, num_channels, image_width, image_height]
  """

  return np.stack([_pad_2d_channel(obs, num) for obs in inp])

순방향 계산

In [55]:
def _compute_output_obs(obs: ndarray,
                        param: ndarray) -> ndarray:
  """
  obs: [channels. img_width, img_height]
  param: [in_channels, out_channels, param_width, param_height]
  """
  assert_dim(obs, 3)
  assert_dim(param, 4)

  param_size = param.shape[2]
  param_mid = param_size // 2
  obs_pad = _pad_2d_channel(obs, param_mid)

  in_channels = param.shape[0]
  out_channels = param.shape[1]
  img_size = obs.shape[1]

  out = np.zeros((out_channels,) + obs.shape[1:])
  for c_in in range(in_channels):
    for c_out in range(out_channels):
      for o_w in range(img_size):
        for o_h in range(img_size):
          for p_w in range(param_size):
            for p_h in range(param_size):
              out[c_out][o_w][o_h] += param[c_in][c_out][p_w][p_h] * obs_pad[c_in][o_w + p_w][o_h + p_h]

  return out

def _output(inp: ndarray,
            param: ndarray) -> ndarray:

  """
  obs: [batch_size, channels, img_width, img_height]
  param: [in_channels, out_channels, param_width, param_height]
  """
  outs = [_compute_output_obs(obs, param) for obs in inp]

  return np.stack(outs)

역방향 계산
1. 입력 기울기를 구할 때는 각 관찰에 대한 기울기를 따로 구한 뒤(이 과정에서 출력 기울기에 패딩을 덧붙인다), 이를 다시 원래대로 쌓는다.
2. 패딩된 출력 기울기는 파라미터 기울기를 구할 때도 사용한다. 그러나 이 경우에는 관찰을 순화하여 각 관찰에서 구한 값으로 파라미터 기울기를 수정한다.

출력 기울기

In [56]:
def _compute_grads_obs(input_obs: ndarray,
                       output_grad_obs: ndarray,
                       param: ndarray) -> ndarray:
  """
  input_obs: [in_channels, img_width, img_height]
  output_grad_obs: [out_channels, img_width, img_height]
  param: [in_channels, out_channels, img_width, img_height]
  """

  input_grad = np.zeros_like(input_obs)
  param_size = param.shape[2]
  param_mid = param_size // 2
  img_size = input_obs.shape[1]
  in_channels = input_obs.shape[0]
  out_channels = param.shape[1]
  output_obs_pad = _pad_2d_channel(output_grad_obs, param_mid)

  for c_in in range(in_channels):
    for c_out in range(out_channels):
      for i_w in range(input_obs.shape[1]):
        for i_h in range(input_obs.shape[2]):
          for p_w in range(param_size):
            for p_h in range(param_size):
              input_grad[c_in][i_w][i_h] += \
              output_obs_pad[c_out][i_w + param_size - p_w - 1][i_h + param_size - p_h - 1] * param[c_in][c_out][p_w][p_h]

  return input_grad

def _input_grad(inp: ndarray,
                output_grad: ndarray,
                param: ndarray) -> ndarray:

  grads = [_compute_grads_obs(inp[i], output_grad[i], param) for i in range(output_grad.shape[0])]

  return np.stack(grads)

파라미터 기울기

In [57]:
def _param_grad(inp: ndarray,
                output_grad: ndarray,
                param: ndarray) -> ndarray:
  """
  inp: [in_channels, img_width, img_height]
  output_grad_obs: [out_channels, img_width, img_height]
  param: [in_channels, out_channels, img_width, img_height]
  """

  param_grad = np.zeros_like(param)
  param_size = param.shape[2]
  param_mid = param_size // 2
  img_size = inp.shape[2]
  in_channels = inp.shape[1]
  out_channels = output_grad.shape[1]

  inp_pad = _pad_conv_input(inp, param_mid)
  img_shape = output_grad.shape[2:]

  for i in range(inp.shape[0]):
    for c_in in range(in_channels):
      for c_out in range(out_channels):
        for o_w in range(img_shape[0]):
          for o_h in range(img_shape[1]):
            for p_w in range(param_size):
              for p_h in range(param_size):
                param_grad[c_in][c_out][p_w][p_h] += \
                inp_pad[i][c_in][o_w + p_w][o_h + p_h] * output_grad[i][c_out][o_w][o_h]

  return param_grad

기울기 검증

In [58]:
cifar_imgs = np.random.randn(10, 3, 32, 32)
cifar_param = np.random.randn(3, 16, 5, 5)

In [59]:
print(np.random.randint(0, cifar_imgs.shape[0]))
print(np.random.randint(0, cifar_imgs.shape[1]))
print(np.random.randint(0, cifar_imgs.shape[2]))
print(np.random.randint(0, cifar_imgs.shape[3]))
print()
print(np.random.randint(0, cifar_param.shape[0]))
print(np.random.randint(0, cifar_param.shape[1]))
print(np.random.randint(0, cifar_param.shape[2]))
print(np.random.randint(0, cifar_param.shape[3]))

9
2
12
4

2
1
3
0


In [60]:
def _compute_output_sum(imgs: ndarray,
                        param: ndarray):
  return _output(imgs, param).sum()

입력 기울기

In [61]:
cifar_imgs_2 = cifar_imgs.copy()
cifar_imgs_2[3][1][2][19] += 1

In [62]:
_compute_output_sum(cifar_imgs_2, cifar_param) - _compute_output_sum(cifar_imgs, cifar_param)

-15.079911141734556

In [63]:
_input_grad(cifar_imgs,
            np.ones((10, 16, 32, 32)),
            cifar_param)[3][1][2][19]

-15.079911141734021

파라미터 기울기

In [64]:
cifar_param_2 = cifar_param.copy()
cifar_param_2[0][8][0][2] += 1

In [65]:
_compute_output_sum(cifar_imgs, cifar_param_2) - _compute_output_sum(cifar_imgs, cifar_param)

-165.40234420503748

In [66]:
_param_grad(cifar_imgs, 
            np.ones((10, 16, 32, 32)),
            cifar_param)[0][8][0][2]

-165.40234420503637

## Conv2DOperation 연산으로 합성곱 신경망 학습하기

1. 앞에서 다룬 Flatten 연산을 구현해야 한다. 모델이 예측을 내릴 때 이 연산을 사용한다.
2. Conv2DLayer 클래스가 Conv2DOperation을 연산으로 사용할 수 있도록 한다.
3. 마지막으로 다른 구성 요소가 좀 더 유용하도록 Conv2DOperation연산의 속도를 개선한다.

In [67]:
# 신경망 프레임워크

class Operation(object):
  """
  신경망 모델의 연산 역할을 하는 기반 클래스
  """
  def __init__(self):
    pass

  def forward(self, input_: ndarray):
    """
    인스턴스 변수 self._input에 입력값을 저장한 다음 self._output() 함수를 호출한다.
    """
    self.input_ = input_
    self.output = self._output()
    return self.output

  def backward(self, output_grad: ndarray) -> ndarray:
    """
    self._input_grad() 함수를 호출한다. 이때 모양이 일치하는지 먼저 확인한다.
    """
    assert_same_shape(self.output, output_grad)
    self.input_grad = self._input_grad(output_grad)

    assert_same_shape(self.input_, self.input_grad)
    return self.input_grad

  def _output(self) -> ndarray:
    """
    Operation을 구현한 모든 구상 클래서는 _output 메서드를 구현해야 한다.
    """
    raise NotImplementedError()

  def _input_grad(self, output_grad: ndarray) -> ndarray:
    """
    Operation을 구현한 모든 구상 클래스는 _input_grad 메서드를 구현해야 한다.
    """
    raise NotImplementedError()

class ParamOperation(Operation):
  """
  파라미터를 갖는 연산
  """

  def __init__(self, param: ndarray) -> ndarray:
    """
    생성자 메서트
    """
    super().__init__()
    self.param = param

  def backward(self, output_grad: ndarray) -> ndarray:
    """
    self._input_grad, self._param_grad를 호출한다.
    이때 ndarray 객체의 모양이 일치하는지 확인한다.
    """
    
    assert_same_shape(self.output, output_grad)

    self.input_grad = self._input_grad(output_grad)
    self.param_grad = self._param_grad(output_grad)

    assert_same_shape(self.input_, self.input_grad)
    assert_same_shape(self.param, self.param_grad)

    return self.input_grad

  def _param_grad(self, output_grad: ndarray) -> ndarray:
    """
    ParamOperation을 구현한 모든 구상 클래스는 _param_grad 메서드를 구현해야 한다.
    """
    raise NotImplementedError()

class WeightMultiply(ParamOperation):
  """
  신경망의 가중치 행렬곱 연산
  """

  def __init__(self, W: ndarray):
    """
    self.param = W로 초기화
    """
    super().__init__(W)

  def _output(self) -> ndarray:
    """
    출력값 계산
    """
    return np.dot(self.input_, self.param)

  def _input_grad(self, output_grad: ndarray) -> ndarray:
    """
    입력에 대한 기울기 계산
    """
    return np.dot(output_grad, np.transpose(self.param, (1, 0)))

  def _param_grad(self, output_grad: ndarray) -> ndarray:
    """
    파라미터에 대한 기울기 계산
    """
    return np.dot(np.transpose(self.input_, (1, 0)), output_grad)

class BiasAdd(ParamOperation):
  """
  편향을 더하는 연산
  """

  def __init__(self, B: ndarray):
    """
    self.param = B로 초기화한다.
    초기화 전에 행렬의 모양을 확인한다.
    """
    assert B.shape[0] == 1

    super().__init__(B)

  def _output(self) -> ndarray:
    """
    출력값 계산
    """
    return self.input_ + self.param

  def _input_grad(self, output_grad: ndarray) -> ndarray:
    """
    입력에 대한 기울기 계산
    """
    return np.ones_like(self.input_) * output_grad

  def _param_grad(self, output_grad: ndarray) -> ndarray:
    """
    파라미터에 대한 기울기 계산
    """
    param_grad = np.ones_like(self.param) * output_grad
    return np.sum(param_grad, axis = 0).reshape(1, param_grad.shape[1])

class Sigmoid(Operation):
  """
  Sigmoid 활성화 함수
  """

  def __init__(self) -> None:
    """ Pass"""
    super().__init__()

  def _output(self) -> ndarray:
    """
    출력값 계산
    """
    return 1.0 / (1.0 + np.exp(-1.0 * self.input_))

  def _input_grad(self, output_grad: ndarray) -> ndarray:
    """
    입력에 대한 기울기 계산
    """
    sigmoid_backward = self.output * (1.0 - self.output)
    input_grad = sigmoid_backward * output_grad
    return input_grad

class Linear(Operation):
  """
  항등 활성화 함수
  """
  def __init__(self) -> None:
    '기반 클래스의 생성자 메서드 실행'
    super().__init__()

  def _output(self) -> ndarray:
    '입력을 그대로 출력'
    return self.input_
  
  def _input_grad(self, output_grad: ndarray) -> ndarray:
    '그대로 출력'
    return output_grad

class Layer(object):
  """
  신경망 모델의 층 역할을 하는 클래스
  """

  def __init__(self, neurons: int):
    """
    뉴런의 개수는 층의 너비에 해당한다.
    """
    self.neurons = neurons
    self.first = True
    self.params: List[ndarray] = []
    self.param_grads: List[ndarray] = []
    self.operations: List[Operation] = []

  def _setup_layer(self, num_in: int) -> None:
    """
    Layer를 구현하는 구상 클래스는 _setup_layer 메서드를 구현해야 한다.
    """
    raise NotImplementedError()

  def forward(self, input_: ndarray) -> ndarray:
    """
    입력값을 각 연산에 순서대로 통과시켜 순방향 계산을 수행한다.
    """
    if self.first:
      self._setup_layer(input_)
      self.first = False

    self.input_ = input_

    for operation in self.operations:
      input_ = operation.forward(input_)

    self.output = input_

    return self.output

  def backward(self, output_grad: ndarray) -> ndarray:
    """
    output_grad를 각 연산에 역순으로 통과시켜 역방향 계산을 수행한다.
    계산하기 전, 행렬의 모양을 검사한다.
    """

    assert_same_shape(self.output, output_grad)

    for operation in reversed(self.operations):
      output_grad = operation.backward(output_grad)

    input_grad = output_grad

    self._param_grads()

    return input_grad

  def _param_grads(self) -> ndarray:
    """
    각 Operation 객체에서 _param_grad 값을 꺼낸다.
    """

    self.param_grads = []
    for operation in self.operations:
      if issubclass(operation.__class__, ParamOperation):      # 상속관계여부 확인 issubclass(자식 클래스, 부모 클래스)
        self.param_grads.append(operation.param_grad)

  def _params(self) -> ndarray:
    """
    각 Operation 객체에서 _params값을 꺼낸다.
    """

    self.params = []
    for operation in self.operations:
      if issubclass(operation.__class__, ParamOperation):
        self.params.append((operation.param))

#### Flatten 연산

In [68]:
class Flatten(Operation):
  def __init__(self):
    super().__init__()

  def _output(self) -> ndarray:
    return self.input.reshape(self.input.shape[0], -1)

  def _input_grad(self, output_grad: ndarray) -> ndarray:
    return output_grad.reshape(self.input.shape)

#### 완성된 합성곱층: Conv2D 클래스

In [69]:
class Conv2D(Layer):

  def __init__(self,
               out_channels: int,
               param_size: int,
               activation: Operation = Sigmoid(),
               flatten: bool = False) -> None:

    super().__init__()
    self.out_channels = out_channels
    self.param_size = param_size
    self.activation = activation
    self.flatten = flatten

  def _setup_layer(self, input_: ndarray) -> ndarray:

    self.params = []
    conv_param = np.random.randn(self.out_channels,
                                 input_.shape[1],
                                 self.param_size,
                                 self.param_size)
    self.param.append(conv_param)

    self.operations = []
    self.operations.append(Conv2D(conv_param))
    self.operations.append(self.activation)

    if self.flatten:
      self.operations.append(Flatten())

    return None

속도 개선을 위한 다른 구현 방법
1. 테스트 데이터에서 filter_height X filter_width 크기의 패치를 image_height X image_width X num_channles개 추출한다.
2. 각 패치와 짝이 맞는 필터로 점곱을 수행한다.
3. 모든 점곱 연산의 결과를 다시 쌓아 모양을 변경해 출력을 구성한다.

#### 실험

데이터 준비

In [70]:
%cd /content/drive/MyDrive/colab/deep_learning_basic/처음_시작하는_딥러닝

/content/drive/MyDrive/colab/deep_learning_basic/처음_시작하는_딥러닝


In [71]:
import lincoln
from lincoln.layers import Dense
from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
from lincoln.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer
from lincoln.utils import mnist
from lincoln.layers import Conv2D

X_train, y_train, X_test, y_test = mnist.load()

In [72]:
%load_ext autoreload
%autoreload 2

In [73]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [74]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [75]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
  train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
  test_labels[i][y_test[i]] = 1

In [76]:
 def calc_accuracy_model(model, test_set):
   return print(f'''모델 검증을 위한 정확도: {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

밑바닥부터 구현한 CNN

In [77]:
# 약 20분 소요
model = NeuralNetwork(
    layers = [Conv2D(out_channels = 32,
                     param_size = 5,
                     dropout = 0.8,
                     weight_init = 'glorot',
                     flatten = True,
                     activation = Tanh()),
              Dense(neurons = 10,
                    activation = Linear())],
                    loss = SoftmaxCrossEntropy(),
                    seed = 20220121
)

trainer = Trainer(model, SGDMomentum(lr = 0.1, momentum = 0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs = 1,
            eval_every = 1,
            seed = 20220121,
            batch_size = 60,
            conv_testing = True)

0 배치 학습 후 손실값은  36.04872314575766
10 배치 학습 후 손실값은  18.11295374600702
20 배치 학습 후 손실값은  9.555350692307362
30 배치 학습 후 손실값은  6.624461311564583
40 배치 학습 후 손실값은  5.970296618173084
50 배치 학습 후 손실값은  4.5842738614953555
60 배치 학습 후 손실값은  4.782127921737016
70 배치 학습 후 손실값은  5.3329694639958
80 배치 학습 후 손실값은  7.009925283806245
90 배치 학습 후 손실값은  9.909754417905422
100 배치 학습 후 손실값은  3.1612291552164917
100 배치 학습 후 검증 데이터에 대한 정확도:  88.49%
110 배치 학습 후 손실값은  2.748140815522369
120 배치 학습 후 손실값은  4.208467145012957
130 배치 학습 후 손실값은  6.216980036716183
140 배치 학습 후 손실값은  4.14472264499166
150 배치 학습 후 손실값은  6.066269842415648
160 배치 학습 후 손실값은  4.835428709045474
170 배치 학습 후 손실값은  4.8354287426811355
180 배치 학습 후 손실값은  2.9236679054445402
190 배치 학습 후 손실값은  4.751381141576223
200 배치 학습 후 손실값은  3.642434542436154
200 배치 학습 후 검증 데이터에 대한 정확도:  88.06%
210 배치 학습 후 손실값은  7.5985308217688
220 배치 학습 후 손실값은  3.9671699981775874
230 배치 학습 후 손실값은  1.6596211439104225
240 배치 학습 후 손실값은  4.144653180017475
250 배치 학습 후 손실값은  3.859690235408356
26

In [78]:
calc_accuracy_model(model, X_test_conv)

모델 검증을 위한 정확도: 89.40%
